In [1]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import scipy as sc
import os
import xarray as xr
import tqdm
from tqdm import tqdm, trange
from scipy import ndimage

This notebook provides code to prepare a matrix for the Kmeans clustering algorithm and perform the clustering

In [92]:
all_means = xr.open_dataarray('/path/to/temperature_data.nc') # this is the daily mean temperature data
cleaned_clusters03 = np.load('/path/to/masked_real_clean_3.npy') # this is the clearned up CAO binary array

In [93]:
mean_temperature = all_means.groupby('time.dayofyear').mean('time')
anomalies = all_means - mean_temperature

In [95]:
labeled_array, num_features = ndimage.label(cleaned_clusters03)

In [81]:

# Create arrays to store the centroids
start_centroids = np.zeros((num_features, 2))  # (y, z) for the first slice
end_centroids = np.zeros((num_features, 2))    # (y, z) for the last slice

# Create an array to store the distances
horizontal_distances = np.zeros(num_features)

# Iterate through each labeled feature
for label in tqdm(range(1, num_features + 1)):  # Start from 1 as 0 is background label
    # Find the indices where the labeled feature is present in the first slice
    indices_start = np.where(labeled_array == label)
    first_slice_index = np.min(indices_start[0])
    first_slice = labeled_array[first_slice_index]
    start_ys_zs = np.where(first_slice == label)

    # Find the centroid in the first slice
    start_centroids[label - 1] = [np.mean(start_ys_zs[0]), np.mean(start_ys_zs[1])]

    # Find the indices where the labeled feature is present in the last slice
    indices_end = np.where(labeled_array == label)
    last_slice_index = np.max(indices_end[0])
    last_slice = labeled_array[last_slice_index]
    end_ys_zs = np.where(last_slice == label)

    # Find the centroid in the last slice
    end_centroids[label - 1] = [np.mean(end_ys_zs[0]), np.mean(end_ys_zs[1])]

    # Calculate the horizontal distance between the starting and ending centroids for each feature
    horizontal_distances[label - 1] = np.linalg.norm(start_centroids[label - 1] - end_centroids[label - 1])

# Now, 'horizontal_distances' contains the horizontal distances between the starting and ending centroids for each labeled feature



 35%|███▌      | 31/88 [00:21<00:39,  1.44it/s]


ValueError: zero-size array to reduction operation minimum which has no identity

In [98]:
indices_start = np.where(labeled_array == 32)
first_slice_index = np.min(indices_start[0])

In [99]:
first_slice_index

2631

In [ ]:
start_x = start_centroids[:, 0]
start_y = start_centroids[:, 1]

In [ ]:
depths = []

for label in tqdm(range(1, num_features + 1)):
    indices = np.where(labeled_array == label)
    if indices[0].size > 0:
        depth = np.max(indices[0]) - np.min(indices[0]) + 1
        depths.append(depth)

In [ ]:
#Normalized anomalies
import numpy as np

std_temps = np.std(anomalies, axis=0)
mean = np.mean(anomalies, axis=0)

normalized_anomalies = (anomalies - mean) / std_temps


In [ ]:

average_normalized_anomalies = np.zeros(num_features)
centroid_max_intensity = np.zeros((num_features, 2))
max_extent2 = []
# Iterate through each labeled feature
for label in tqdm(range(1, num_features + 1)):  # Start from 1 as 0 is background label
    # Find the indices where the labeled feature is present
    indices = np.where(labeled_array == label)
    extent = []
    for i in np.unique(indices[0]):
        extent.append(np.sum(labeled_array[i]) / label)
        max_extent = np.argmax(extent)
        max_extent_i = max_extent + np.unique(indices[0])[0]
    max_extent2.append(np.max(extent))
    intense_slice = labeled_array[max_extent_i]
    ys_zs = np.where(intense_slice == label)
    centroid_max_intensity[label - 1] = [np.mean(ys_zs[0]), np.mean(ys_zs[1])]

    # Extract the normalized anomalies for the labeled feature using the unique indices
    for i in range(len(ys_zs[0])):
        anomalies_for_feature = []
        anomalies_for_feature.append(anomalies[max_extent_i][ys_zs[0][i]][ys_zs[1][i]])


    # Calculate the average normalized anomaly for the labeled feature
    average_normalized_anomaly = np.mean(anomalies_for_feature)

    # Store the result in the array
    average_normalized_anomalies[label - 1] = average_normalized_anomaly

# Now, 'average_normalized_anomalies' contains the average normalized anomalies for each labeled feature


In [ ]:
np.mean(max_extent2)

In [ ]:
mean_temps = np.mean(average_normalized_anomalies)
std_temps = np.std(average_normalized_anomalies)
normalized_anomalies = (average_normalized_anomalies - mean_temps) / std_temps

mean_extent = np.mean(max_extent2)
std_extent = np.std(max_extent2)
normalized_extent = (max_extent2 - mean_extent) / std_extent

mean_distances = np.mean(horizontal_distances)
std_distances = np.std(horizontal_distances)
normalized_distances = (horizontal_distances - mean_distances) / std_distances

In [ ]:
np.save('/glade/work/zsolecki/arrays/normalized_anomalies.npy', normalized_anomalies)
np.save('/glade/work/zsolecki/arrays/normalized_distances.npy', normalized_distances)
np.save('/glade/work/zsolecki/arrays/normalized_extent.npy', normalized_extent)

In [82]:
centroid_x = centroid_max_intensity[:, 0]
centroid_y = centroid_max_intensity[:, 1]

In [ ]:
mean_x = np.mean(centroid_x)
std_x = np.std(centroid_x)
normalized_x = (centroid_x - mean_x) / std_x

mean_y = np.mean(centroid_y)
std_y = np.std(centroid_y)
normalized_y = (centroid_y - mean_y) / std_y

In [83]:
np.save('/glade/work/zsolecki/CAO_project/arrays/normalized_x_new.npy', centroid_x)
np.save('/glade/work/zsolecki/CAO_project/arrays/notmalized_y_new.npy', centroid_y)

In [ ]:
speed = horizontal_distances / depths

In [ ]:
mean_speed = np.mean(speed)
std_speed = np.std(speed)
normalized_speed = (speed - mean_speed) / std_speed
np.save('/glade/work/zsolecki/arrays/normalized_speed.npy', normalized_speed)

In [ ]:
volumes = []
for label in tqdm(range(1, num_features + 1)):
    volume = np.sum(labeled_array == label)
    volumes.append(volume)

In [ ]:
mean_volumes = np.mean(volumes)
std_volumes = np.std(volumes)
normalized_volumes = (volumes - mean_volumes) / std_volumes

In [ ]:
np.save('/glade/work/zsolecki/arrays/normalized_volumes.npy', normalized_volumes)

In [ ]:
feature_matrix = np.column_stack((
    normalized_volumes,
    normalized_distances,
    normalized_extent,
    normalized_anomalies,
    normalized_speed,
    normalized_x,
    normalized_y
))

np.save('../data/feature_matrix.npy', feature_matrix)

In [ ]:
from sklearn.cluster import KMeans

optimal_k = 5  # Replace with your optimal number of clusters

# Initialize KMeans with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)

# Fit KMeans to your feature matrix
cluster_labels = kmeans.fit_predict(feature_matrix)

# Add the cluster labels as a new column in your feature matrix
feature_matrix_with_clusters = np.column_stack((feature_matrix, cluster_labels))

In [ ]:
clusters = feature_matrix_with_clusters[:, 7]

In [ ]:
indices_0 = np.where(clusters == 0)[0]
indices_1 = np.where(clusters == 1)[0]
indices_2 = np.where(clusters == 2)[0]
indices_3 = np.where(clusters == 3)[0]
indices_4 = np.where(clusters == 4)[0]

In [ ]:
type(volumes)

In [ ]:
volume_0 = np.array(volumes)[indices_0]
volume_1 = np.array(volumes)[indices_1]
volume_2 = np.array(volumes)[indices_2]
volume_3 = np.array(volumes)[indices_3]
volume_4 = np.array(volumes)[indices_4]

In [ ]:
extent_0 = np.array(max_extent2)[indices_0]
extent_1 = np.array(max_extent2)[indices_1]
extent_2 = np.array(max_extent2)[indices_2]
extent_3 = np.array(max_extent2)[indices_3]
extent_4 = np.array(max_extent2)[indices_4]

In [ ]:
speed_0 = np.array(speed)[indices_0]
speed_1 = np.array(speed)[indices_1]
speed_2 = np.array(speed)[indices_2]
speed_3 = np.array(speed)[indices_3]
speed_4 = np.array(speed)[indices_4]

In [ ]:
anomalies_0 = np.array(average_normalized_anomalies)[indices_0]
anomalies_1 = np.array(average_normalized_anomalies)[indices_1]
anomalies_2 = np.array(average_normalized_anomalies)[indices_2]
anomalies_3 = np.array(average_normalized_anomalies)[indices_3]
anomalies_4 = np.array(average_normalized_anomalies)[indices_4]

In [ ]:
distances_0 = np.array(horizontal_distances)[indices_0]
distances_1 = np.array(horizontal_distances)[indices_1]
distances_2 = np.array(horizontal_distances)[indices_2]
distances_3 = np.array(horizontal_distances)[indices_3]
distances_4 = np.array(horizontal_distances)[indices_4]

In [ ]:
x_0 = np.array(centroid_x)[indices_0]
x_1 = np.array(centroid_x)[indices_1]
x_2 = np.array(centroid_x)[indices_2]
x_3 = np.array(centroid_x)[indices_3]
x_4 = np.array(centroid_x)[indices_4]

In [ ]:
y_0 = np.array(centroid_y)[indices_0]
y_1 = np.array(centroid_y)[indices_1]
y_2 = np.array(centroid_y)[indices_2]
y_3 = np.array(centroid_y)[indices_3]
y_4 = np.array(centroid_y)[indices_4]

In [ ]:
matrix_0 = np.column_stack((
    volume_0,
    distances_0,
    extent_0,
    anomalies_0,
    speed_0,
    x_0,
    y_0, 
    indices_0
))

In [ ]:
matrix_1 = np.column_stack((
    volume_1,
    distances_1,
    extent_1,
    anomalies_1,
    speed_1,
    x_1,
    y_1,
    indices_1
))

In [ ]:
matrix_2 = np.column_stack((
    volume_2,
    distances_2,
    extent_2,
    anomalies_2,
    speed_2,
    x_2,
    y_2,
    indices_2
))

In [ ]:
matrix_3 = np.column_stack((
    volume_3,
    distances_3,
    extent_3,
    anomalies_3,
    speed_3,
    x_3,
    y_3,
    indices_3
))

In [ ]:
matrix_4 = np.column_stack((
    volume_4,
    distances_4,
    extent_4,
    anomalies_4,
    speed_4,
    x_4,
    y_4,
    indices_4
))

In [ ]:
np.save('/glade/work/zsolecki/arrays/matrix_0.npy', matrix_0)
np.save('/glade/work/zsolecki/arrays/matrix_1.npy', matrix_1)
np.save('/glade/work/zsolecki/arrays/matrix_2.npy', matrix_2)
np.save('/glade/work/zsolecki/arrays/matrix_3.npy', matrix_3)
np.save('/glade/work/zsolecki/arrays/matrix_4.npy', matrix_4)